# Meter Extraction
Goal: Extract distinct meters and relevant attributes from PVACD Access DB

In [1]:
import pyodbc
import pandas as pd

In [2]:
#Set up PyODBC connection to Access database
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\Users\clmbn\NMWaterData\PVACDdb\PVACD AMS Jan2023.accdb;'
    )
cnxn = pyodbc.connect(conn_str)

### Data Import
Source: 
tblMeter - It appears that tblMeter is mostly tracking maintenance while tblMeter is mostly unique meter serial numbers.

Fields:
- sTag, sSize, sBrand, sContact, sSerialNumber, sRANumber, sTRSS, sLatLong, sModel, sStatus

In [3]:
qry = '''
    SELECT DISTINCT sSerialNumber, sSize, sBrand, sModel, sContact, sTag, sRANumber, sTRSS, sLatLong, sStatus 
    FROM tblMeter
    WHERE sSerialNumber IS NOT NULL AND sSerialNumber <> ''
'''
meter_df = pd.read_sql_query(qry, cnxn)
meter_df.head()

C:\Users\clmbn\AppData\Local\Temp\ipykernel_22992\2460389883.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  meter_df = pd.read_sql_query(qry, cnxn)


,sSerialNumber,sSize,sBrand,sModel,sContact,sTag,sRANumber,sTRSS,sLatLong,sStatus
0,88-8-817,8.0,MCCROMETER,308,HALDEMAN W.T. ESTATE,None,763;1180-S,17.26.14.133,N32* 50' 08.9'' W104* 21' 38.0'',S
1,94-8-2251,8.0,MCCROMETER,308,DAVID AND TRACEY CORN,221C4,2417 POD 2,8.24.28.413,,I
2,00-10-6922,10.0,MCCROMETER,310,AG SERVICES,None,,"ROSWELL, NM",,S
3,00-10-6923,10.0,MCCROMETER,310,N.M.S.U. EXPER STAT.,20138,827,18.26.16.211,N32*-45' 16.4'' W104*-23' 11.1'',I
4,00-10-6924,10.0,MCCROMETER,310,WAIDE FARMS,None,None,None,None,S


### Data Cleaning
- Remove duplicates

In [4]:
#Drop duplicate serial numbers from meter_df
meter_df.drop_duplicates(subset='sSerialNumber', keep='first', inplace=True)

### Prepare data for new database